In [ ]:
!pip install -r "D:\hang_test\04.01\kcbert\ratsnlp\requirements.txt"

In [1]:
%cd "C:\Users\coms\Desktop\04.07\kcbert\ratsnlp\ratsnlp\nlpbook"

C:\Users\coms\Desktop\04.07\kcbert\ratsnlp\ratsnlp\nlpbook


In [ ]:
# plumber 사용하기 

In [2]:
import torch
from classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",  #프리트레인 마친 언어모델의 이름
    downstream_corpus_name="data", # 다운스트림 데이터의 이름
    downstream_model_dir="C:/Users/coms/Desktop/04.07/kcbert/result",  # 파인튜닝된 모델의 체크포인트가 저장될 위치. 
    downstream_corpus_root_dir =  "C:/Users/coms/Desktop/04.07/kcbert/result/Korpora",
    batch_size=32 if torch.cuda.is_available() else 4, #TPU로 설정했다면 4
    learning_rate=5e-5,   
    max_seq_length=128,  # 패딩과 같은 의미 / 이보다 긴문장은 자르고 짧은 문장은 padding을 진행한다. 
    epochs=1,
    tpu_cores=0 if torch.cuda.is_available() else 8, # GPU면 TPU 코어수 0 TPU면 8
    seed=7,
)

In [3]:
args.downstream_corpus_root_dir

'C:/Users/coms/Desktop/04.07/kcbert/result/Korpora'

In [4]:
from utils import set_seed
set_seed(args)

set seed: 7


In [5]:
from utils import set_logger
set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='data', downstream_corpus_root_dir='C:/Users/coms/Desktop/04.07/kcbert/result/Korpora', downstream_model_dir='C:/Users/coms/Desktop/04.07/kcbert/result', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=16, fp16=False, tpu_cores=0)


In [ ]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir= args.downstream_corpus_root_dir,
    force_download=True,
)

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [7]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from classification import NsmcCorpus, ClassificationDataset
from data_utils import data_collator
corpus = NsmcCorpus()    #“문장(영화 리뷰) + 레이블(긍정, 부정)”

## 데이터 토크나이저 하기 : 모델이 학습할 수 있는 형태로 가공
train_dataset = ClassificationDataset(  
    args=args,
    corpus=corpus,#“문장(영화 리뷰) + 레이블(긍정, 부정)”
    tokenizer=tokenizer, #모델이 학습할 수 있는 형태로 가공
    mode="train",
)

# 전체 인스턴스 가운데 배치크기 만큼 뽑아 배치형태로 가공하기 
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False), # 배치를 만들 대 전체 인스턴스 중 배치사이즈 수 만큼 비복원 추출을 한다. 
    collate_fn=data_collator, # 리스트 형의 ClassificationDataset를 tensor형으로 바꾼다. 
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\cached_train_BertTokenizer_128_data_document-classification [took 0.004 s]


In [8]:
from data_utils import data_collator
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset), # 평가용 데이터 로더 구축. batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 합니다. #학습은 랜덤 평가는 순서대로
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file C:/Users/coms/Desktop/04.07/kcbert/result/Korpora\data\cached_test_BertTokenizer_128_data_document-classification [took 0.001 s]


In [ ]:
# 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

In [ ]:
from classification import ClassificationTask
task = ClassificationTask(model, args)

In [ ]:
from trainer import get_trainer
trainer = get_trainer(args)

In [ ]:
from trainer import get_trainer

trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 토크나이저 초기화하기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
from classification import ClassificationDeployArguments
from transformers import BertConfig, BertForSequenceClassification
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",  
    downstream_model_dir="C:/Users/coms/Desktop/04.07/kcbert/result", 
    max_seq_length=128,  
)



fine_tuned_model_ckpt = torch.load(   
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

model = BertForSequenceClassification(pretrained_model_config)


model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

model.eval()

In [ ]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  #inputs를 파이토치 텐서로 바꾼후 모델 계산하기
        prob = outputs.logits.softmax(dim=1)  ## outputs.logit은 soft-max함수에 넣기 전 로짓(logit)형태이다. 
        zero_prob = round(prob[0][0].item(), 4) ## 긍정/부정일 확률을 소수점 4자리로 반올림
        one_prob = round(prob[0][1].item(), 4)
        two_prob = round(prob[0][2].item(), 4)
        three_prob = round(prob[0][3].item(), 4)
        four_prob = round(prob[0][4].item(), 4)
        five_prob = round(prob[0][5].item(), 4)
        six_prob = round(prob[0][6].item(), 4)
    return prob

In [ ]:
trial_text = pd.read_csv

In [12]:
import csv
data_fpath = 'C:\\Users\\coms\\Desktop\\04.07\\kcbert\\result\\Korpora\\data\\ratings_train.txt'


lines = list(csv.reader(open(data_fpath, "r", encoding="utf-8"), delimiter="\t", quotechar='"'))

examples = []
for (i, line) in enumerate(lines):
    if i == 0:
        continue
    label = line[0].split(',')[0]
    text_a = line[0].split(',')[1]

    examples.append(ClassificationExample(text_a=text_a, text_b=None, label=label))
print(examples[0])



NameError: name 'ClassificationExample' is not defined